In [1]:
import numpy as np
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler, normalize
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.svm import SVC
import re
from sklearn import svm, datasets
from sklearn.preprocessing import LabelEncoder
import time
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.decomposition import PCA

# Data importing

In [90]:
# edit this with you data path
url = "../input/human-activity-recognition-with-smartphones/train.csv"
train_set = pd.read_csv(url)
url = "../input/human-activity-recognition-with-smartphones/test.csv"
test_set = pd.read_csv(url)



# Eliminating Nan value Records and the column that indicates the subject ID (irrelevant variable)

In [95]:
train_set = train_set.dropna()
del train_set["subject"]
test_set = test_set.dropna()
del test_set["subject"]


# EDA

In [96]:
print("training set size")
print(train_set.shape)
print("testing set size")
print(test_set.shape)
print('training data statistical description')
print(train_set.describe())
print('testing data statistical description')
print(test_set.describe())
print("variables type")
print(train_set.dtypes)




We could conclude from the statistics above that almost all features are bounded within the range [-1,1]. The standart deviation is significant enough to  except for some some variables like tBodyAcc-mean()-X  tBodyAcc-mean()-Y  tBodyAcc-mean()-Z (below 0.02), this could lead us to think that these 3 variable doesn't contribute much to explain the activity type but we won't assume so for now. Let's remember that all variable values are limited between 1 -1 (a small interval), therefore a low std of the order 0.0 would only mean that there's very little to no variance around the mean.

# Target variable probability distribution

In [5]:
Activity = ["WALKING","WALKING_UPSTAIRS","WALKING_DOWNSTAIRS","SITTING", "STANDING", "LAYING"]
objects = Activity
activities_frequencies = (train_set.groupby('Activity').size()/7352).tolist()
y_pos = np.arange(len(objects))

plt.figure(figsize=(20, 6))
plt.bar(y_pos, activities_frequencies, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Distribution % ')
plt.title("Distribution of activities recorded for the training data")

In [6]:
activities_frequencies = (test_set.groupby('Activity').size()/2947).tolist()
y_pos = np.arange(len(objects))

plt.figure(figsize=(20, 6))
plt.bar(y_pos, activities_frequencies, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Distribution % ')
plt.title("Distribution of activities recorded for the testing data")

The training and testing activity distribution are similar, the activities proportions in the dataset are all (almost) equal, which means there will be no room for bias. The dataset was put together properly.

# Bivariate analysis/feature selection

In [ ]:
print(train_set.columns)

We replace features that are highly correlated with each other (corr coefficient >= 0.9) with their PCA representation. We will keep an eye on the diagonal of the correlation matrix and watch subsets of features that are correlated with one another.

In [7]:
corrMatrix = train_set[['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z','tBodyAcc-std()-X', 'tBodyAcc-std()-Y', 'tBodyAcc-std()-Z','tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y', 'tBodyAcc-mad()-Z','tBodyAcc-max()-X','tBodyAcc-max()-Y','tBodyAcc-max()-Z']].corr()
plt.figure(figsize=(20, 6))
sns.heatmap(corrMatrix, annot=True)
plt.show()


In [9]:
corrMatrix = train_set[list(train_set.columns)[0:50]].corr()
plt.figure(figsize=(20, 15))
sns.heatmap(corrMatrix, annot=True)
plt.show()


In [44]:
corrMatrix = train_set[list(train_set.columns)[50:100]].corr()
plt.figure(figsize=(20, 20))
sns.heatmap(corrMatrix, annot=True)
plt.show()

In [11]:
corrMatrix = train_set[list(train_set.columns)[100:150]].corr()
plt.figure(figsize=(20, 20))
sns.heatmap(corrMatrix, annot=True)
plt.show()

In [12]:
corrMatrix = train_set[list(train_set.columns)[150:200]].corr()
plt.figure(figsize=(20, 20))
sns.heatmap(corrMatrix, annot=True)
plt.show()

In [41]:
corrMatrix = train_set[list(train_set.columns)[200:250]].corr()
plt.figure(figsize=(20, 20))
sns.heatmap(corrMatrix, annot=True)
plt.show()

In [45]:
corrMatrix = train_set[list(train_set.columns)[250:300]].corr()
plt.figure(figsize=(20, 20))
sns.heatmap(corrMatrix, annot=True)
plt.show()

In [15]:
corrMatrix = train_set[list(train_set.columns)[300:350]].corr()
plt.figure(figsize=(20, 20))
sns.heatmap(corrMatrix, annot=True)
plt.show()

In [16]:
corrMatrix = train_set[list(train_set.columns)[350:400]].corr()
plt.figure(figsize=(20, 20))
sns.heatmap(corrMatrix, annot=True)
plt.show()

In [17]:
corrMatrix = train_set[list(train_set.columns)[400:450]].corr()
plt.figure(figsize=(20, 20))
sns.heatmap(corrMatrix, annot=True)
plt.show()

In [97]:


features_to_remove = ['tBodyAcc-std()-X', 'tBodyAcc-std()-Y', 'tBodyAcc-std()-Z','tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y', 'tBodyAcc-mad()-Z','tBodyAcc-max()-X','tBodyAcc-max()-Y','tBodyAcc-max()-Z']


pca = PCA(n_components=1)
pca.fit(np.array(train_set[features_to_remove]))
print("variance explained by new feature tBodyAcc : ",pca.explained_variance_ratio_)

y = pca.transform(np.array(train_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'tBodyAcc'})
train_set["tBodyAcc"] = new_component 

pca.fit(np.array(test_set[features_to_remove]))


y = pca.transform(np.array(test_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'tBodyAcc'})
test_set["tBodyAcc"] = new_component 

for feature in features_to_remove:
    if feature in train_set.columns:
        del train_set[feature]
    if feature in test_set.columns:
        del test_set[feature]


In [98]:
features_to_remove = ["tBodyAcc-min()-X","tBodyAcc-min()-Y","tBodyAcc-min()-Z"]

pca = PCA(n_components=1)
pca.fit(np.array(train_set[features_to_remove]))
print("variance explained by new feature tBodyAcc min X Y Z",pca.explained_variance_ratio_)

y = pca.transform(np.array(train_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'tBodyAcc min X Y Z'})
train_set["tBodyAcc min X Y Z"] = new_component 

pca.fit(np.array(test_set[features_to_remove]))

y = pca.transform(np.array(test_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'tBodyAcc min X Y Z'})
test_set["tBodyAcc min X Y Z"] = new_component 

for feature in features_to_remove:
    if feature in train_set.columns:
        del train_set[feature]
    if feature in test_set.columns:
        del test_set[feature]

In [99]:


features_to_remove = ["tBodyGyro-arCoeff()-X,1","tBodyGyro-arCoeff()-X,2"]

pca = PCA(n_components=1)
pca.fit(np.array(train_set[features_to_remove]))
print("variance explained by new feature tBodyGyro-arCoeff()-X,1,2 : ",pca.explained_variance_ratio_)

y = pca.transform(np.array(train_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'tBodyGyro-arCoeff()-X,1,2'})
train_set["tBodyGyro-arCoeff()-X,1,2"] = new_component 

pca.fit(np.array(test_set[features_to_remove]))
y = pca.transform(np.array(test_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'tBodyGyro-arCoeff()-X,1,2'})
test_set["tBodyGyro-arCoeff()-X,1,2"] = new_component 

for feature in features_to_remove:
    if feature in train_set.columns:
        del train_set[feature]
    if feature in test_set.columns:
        del test_set[feature]
        
features_to_remove = ["tBodyGyro-arCoeff()-Z,1","tBodyGyro-arCoeff()-Z,2"]

pca = PCA(n_components=1)
pca.fit(np.array(train_set[features_to_remove]))
print("variance explained by new feature tBodyGyro-arCoeff()-Z,1,2 : ",pca.explained_variance_ratio_)

y = pca.transform(np.array(train_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'tBodyGyro-arCoeff()-Z,1,2'})
train_set["tBodyGyro-arCoeff()-Z,1,2"] = new_component 

pca.fit(np.array(test_set[features_to_remove]))
y = pca.transform(np.array(test_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'tBodyGyro-arCoeff()-Z,1,2'})
test_set["tBodyGyro-arCoeff()-Z,1,2"] = new_component 

for feature in features_to_remove:
    if feature in train_set.columns:
        del train_set[feature]
    if feature in test_set.columns:
        del test_set[feature]

features_to_remove = ["tBodyGyro-std()-Y","tBodyGyro-std()-X","tBodyGyro-std()-Z"]

pca = PCA(n_components=1)
pca.fit(np.array(train_set[features_to_remove]))
print("variance explained by new feature tBodyGyro-std()",pca.explained_variance_ratio_)

y = pca.transform(np.array(train_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'tBodyGyro-std()'})
train_set["tBodyGyro-std()"] = new_component 

pca.fit(np.array(test_set[features_to_remove]))
y = pca.transform(np.array(test_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'tBodyGyro-std()'})
test_set["tBodyGyro-std()"] = new_component 

for feature in features_to_remove:
    if feature in train_set.columns:
        del train_set[feature]
    if feature in test_set.columns:
        del test_set[feature]
        

In [100]:
features_to_remove = ["fBodyAcc-skewness()-X","fBodyAcc-kurtosis()-X"]

pca = PCA(n_components=1)
pca.fit(np.array(train_set[features_to_remove]))
print("variance explained by new feature fBodyAcc-skewness-kurtosis()-X : ",pca.explained_variance_ratio_)


y = pca.transform(np.array(train_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'fBodyAcc-skewness-kurtosis()-X'})
train_set["fBodyAcc-skewness-kurtosis()-X"] = new_component 

pca.fit(np.array(test_set[features_to_remove]))

y = pca.transform(np.array(test_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'fBodyAcc-skewness-kurtosis()-X'})
test_set["fBodyAcc-skewness-kurtosis()-X"] = new_component 

for feature in features_to_remove:
    if feature in train_set.columns:
        del train_set[feature]
    if feature in test_set.columns:
        del test_set[feature]

features_to_remove = ["fBodyAcc-skewness()-Y","fBodyAcc-kurtosis()-Y"]

pca = PCA(n_components=1)
pca.fit(np.array(train_set[features_to_remove]))
print("variance explained by new feature fBodyAcc-skewness-kurtosis()-Y : ",pca.explained_variance_ratio_)


y = pca.transform(np.array(train_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'fBodyAcc-skewness-kurtosis()-Y'})
train_set["fBodyAcc-skewness-kurtosis()-Y"] = new_component 

pca.fit(np.array(test_set[features_to_remove]))

y = pca.transform(np.array(test_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'fBodyAcc-skewness-kurtosis()-Y'})
test_set["fBodyAcc-skewness-kurtosis()-Y"] = new_component 

for feature in features_to_remove:
    if feature in train_set.columns:
        del train_set[feature]
    if feature in test_set.columns:
        del test_set[feature]
        
features_to_remove = ["fBodyAcc-mean()-X","fBodyAcc-mean()-Y","fBodyAcc-mean()-Z"]

pca = PCA(n_components=1)
pca.fit(np.array(train_set[features_to_remove]))
print("variance explained by new feature fBodyAcc-mean() : ",pca.explained_variance_ratio_)


y = pca.transform(np.array(train_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'fBodyAcc-mean()'})
train_set["fBodyAcc-mean()"] = new_component 

pca.fit(np.array(test_set[features_to_remove]))

y = pca.transform(np.array(test_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'fBodyAcc-mean()'})
test_set["fBodyAcc-mean()"] = new_component 

for feature in features_to_remove:
    if feature in train_set.columns:
        del train_set[feature]
    if feature in test_set.columns:
        del test_set[feature]
        


features_to_remove = ["fBodyAcc-skewness()-Z","fBodyAcc-kurtosis()-Z"]

pca = PCA(n_components=1)
pca.fit(np.array(train_set[features_to_remove]))
print("variance explained by new feature fBodyAcc-skewness-kurtosis()-Z : ",pca.explained_variance_ratio_)


y = pca.transform(np.array(train_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'fBodyAcc-skewness-kurtosis()-Z'})
train_set["fBodyAcc-skewness-kurtosis()-Z"] = new_component 

pca.fit(np.array(test_set[features_to_remove]))

y = pca.transform(np.array(test_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'fBodyAcc-skewness-kurtosis()-Z'})
test_set["fBodyAcc-skewness-kurtosis()-Z"] = new_component 

for feature in features_to_remove:
    if feature in train_set.columns:
        del train_set[feature]
    if feature in test_set.columns:
        del test_set[feature]


In [101]:
features_to_remove = ["tBodyGyroMag-mean()","tBodyGyroMag-std()","tBodyGyroMag-min()","tBodyGyroMag-max()","tBodyGyroMag-sma()","tBodyGyroMag-mad()"]

pca = PCA(n_components=1)
pca.fit(np.array(train_set[features_to_remove]))
print("variance explained by new feature tBodyGyroMag : ",pca.explained_variance_ratio_)

y = pca.transform(np.array(train_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'tBodyGyroMag'})
train_set["tBodyGyroMag"] = new_component 

pca.fit(np.array(test_set[features_to_remove]))
y = pca.transform(np.array(test_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'tBodyGyroMag'})
test_set["tBodyGyroMag"] = new_component 

for feature in features_to_remove:
    if feature in train_set.columns:
        del train_set[feature]
    if feature in test_set.columns:
        del test_set[feature]

features_to_remove = ["tBodyAccJerkMag-mean()","tBodyAccJerkMag-std()","tBodyAccJerkMag-min()","tBodyAccJerkMag-max()","tBodyAccJerkMag-sma()","tBodyAccJerkMag-mad()"]

pca = PCA(n_components=1)
pca.fit(np.array(train_set[features_to_remove]))
print("variance explained by new feature tBodyAccJerkMag : ",pca.explained_variance_ratio_)

y = pca.transform(np.array(train_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'tBodyAccJerkMag'})
train_set["tBodyAccJerkMag"] = new_component 

pca.fit(np.array(test_set[features_to_remove]))
y = pca.transform(np.array(test_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'tBodyAccJerkMag'})
test_set["tBodyAccJerkMag"] = new_component 

for feature in features_to_remove:
    if feature in train_set.columns:
        del train_set[feature]
    if feature in test_set.columns:
        del test_set[feature]


In [102]:
features_to_remove = ["fBodyAcc-entropy()-X","fBodyAcc-entropy()-Y","fBodyAcc-entropy()-Z"]

pca = PCA(n_components=1)
pca.fit(np.array(train_set[features_to_remove]))
print("variance explained by new feature fBodyAcc-entropy : ",pca.explained_variance_ratio_)


y = pca.transform(np.array(train_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'fBodyAcc-entropy'})
train_set["fBodyAcc-entropy"] = new_component 

pca.fit(np.array(test_set[features_to_remove]))

y = pca.transform(np.array(test_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'fBodyAcc-entropy'})
test_set["fBodyAcc-entropy"] = new_component 

for feature in features_to_remove:
    if feature in train_set.columns:
        del train_set[feature]
    if feature in test_set.columns:
        del test_set[feature]
        

In [58]:
features_to_remove = ["tBodyGyroJerk-entropy()-X","tBodyGyroJerk-entropy()-Y","tBodyGyroJerk-entropy()-Z"]

pca = PCA(n_components=1)
pca.fit(np.array(train_set[features_to_remove]))
print("variance explained by new feature tBodyGyroJerk-entropy() : ",pca.explained_variance_ratio_)


y = pca.transform(np.array(train_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'tBodyGyroJerk-entropy()'})
train_set["tBodyGyroJerk-entropy()"] = new_component 

pca.fit(np.array(test_set[features_to_remove]))

y = pca.transform(np.array(test_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'tBodyGyroJerk-entropy()'})
test_set["tBodyGyroJerk-entropy()"] = new_component 

for feature in features_to_remove:
    if feature in train_set.columns:
        del train_set[feature]
    if feature in test_set.columns:
        del test_set[feature]
        
features_to_remove = ["tBodyGyroJerk-iqr()-X","tBodyGyroJerk-iqr()-Y","tBodyGyroJerk-iqr()-Z"]

pca = PCA(n_components=1)
pca.fit(np.array(train_set[features_to_remove]))
print("variance explained by new feature tBodyGyroJerk-iqr() : ",pca.explained_variance_ratio_)


y = pca.transform(np.array(train_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'tBodyGyroJerk-iqr()'})
train_set["tBodyGyroJerk-iqr()"] = new_component 

pca.fit(np.array(test_set[features_to_remove]))

y = pca.transform(np.array(test_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'tBodyGyroJerk-iqr()'})
test_set["tBodyGyroJerk-iqr()"] = new_component 

for feature in features_to_remove:
    if feature in train_set.columns:
        del train_set[feature]
    if feature in test_set.columns:
        del test_set[feature]

In [103]:
features_to_remove = ["fBodyBodyAccJerkMag-mean()","fBodyBodyAccJerkMag-std()","fBodyBodyAccJerkMag-mad()","fBodyBodyAccJerkMag-max()","fBodyBodyAccJerkMag-min()","fBodyBodyAccJerkMag-sma()","fBodyBodyAccJerkMag-energy()","fBodyBodyAccJerkMag-iqr()","fBodyBodyAccJerkMag-entropy()"]


pca = PCA(n_components=1)
pca.fit(np.array(train_set[features_to_remove]))
print("variance explained by new feature fBodyBodyAccJerkMag : ",pca.explained_variance_ratio_)

y = pca.transform(np.array(train_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'fBodyBodyAccJerkMag'})
train_set["fBodyBodyAccJerkMag"] = new_component 


y = pca.transform(np.array(test_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'fBodyBodyAccJerkMag'})
test_set["fBodyBodyAccJerkMag"] = new_component 

for feature in features_to_remove:
    if feature in train_set.columns:
        del train_set[feature]
    if feature in test_set.columns:
        del test_set[feature]


In [104]:
features_to_remove = ["tBodyAccMag-mean()","tBodyAccMag-std()","tBodyAccMag-mad()","tBodyAccMag-max()","tBodyAccMag-min()","tBodyAccMag-sma()","tBodyAccMag-energy()","tBodyAccMag-iqr()","tBodyAccMag-entropy()"]


pca = PCA(n_components=1)
pca.fit(np.array(train_set[features_to_remove]))
print("variance explained by new feature tBodyAccMag : ",pca.explained_variance_ratio_)

y = pca.transform(np.array(train_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'tBodyAccMag'})
train_set["tBodyAccMag"] = new_component 


y = pca.transform(np.array(test_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'tBodyAccMag'})
test_set["tBodyAccMag"] = new_component 

for feature in features_to_remove:
    if feature in train_set.columns:
        del train_set[feature]
    if feature in test_set.columns:
        del test_set[feature]


In [105]:

features_to_remove = ["tGravityAccMag-mean()","tGravityAccMag-std()","tGravityAccMag-mad()","tGravityAccMag-max()","tGravityAccMag-min()","tGravityAccMag-sma()","tGravityAccMag-energy()","tGravityAccMag-iqr()","tGravityAccMag-entropy()"]


pca = PCA(n_components=1)
pca.fit(np.array(train_set[features_to_remove]))
print("variance explained by new feature tGravityAccMag : ",pca.explained_variance_ratio_)

y = pca.transform(np.array(train_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'tGravityAccMag'})
train_set["tGravityAccMag"] = new_component 


y = pca.transform(np.array(test_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'tGravityAccMag'})
test_set["tGravityAccMag"] = new_component 

for feature in features_to_remove:
    if feature in train_set.columns:
        del train_set[feature]
    if feature in test_set.columns:
        del test_set[feature]

In [106]:
features_to_remove = ["fBodyGyro-entropy()-X","fBodyGyro-entropy()-Y","fBodyGyro-entropy()-Z"]

pca = PCA(n_components=1)
pca.fit(np.array(train_set[features_to_remove]))
print("variance explained by new feature fBodyGyro-entropy() : ",pca.explained_variance_ratio_)


y = pca.transform(np.array(train_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'fBodyGyro-entropy()'})
train_set["fBodyGyro-entropy()"] = new_component 

pca.fit(np.array(test_set[features_to_remove]))

y = pca.transform(np.array(test_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'fBodyGyro-entropy()'})
test_set["fBodyGyro-entropy()"] = new_component 

for feature in features_to_remove:
    if feature in train_set.columns:
        del train_set[feature]
    if feature in test_set.columns:
        del test_set[feature]

In [107]:
features_to_remove = ["fBodyAccJerk-entropy()-X","fBodyAccJerk-entropy()-Y","fBodyAccJerk-entropy()-Z"]

pca = PCA(n_components=1)
pca.fit(np.array(train_set[features_to_remove]))
print("variance explained by new feature fBodyAccJerk-entropy() : ",pca.explained_variance_ratio_)


y = pca.transform(np.array(train_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'fBodyAccJerk-entropy()'})
train_set["fBodyAccJerk-entropy()"] = new_component 

pca.fit(np.array(test_set[features_to_remove]))

y = pca.transform(np.array(test_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'fBodyAccJerk-entropy()'})
test_set["fBodyAccJerk-entropy()"] = new_component 

for feature in features_to_remove:
    if feature in train_set.columns:
        del train_set[feature]
    if feature in test_set.columns:
        del test_set[feature]

In [108]:
features_to_remove = ["fBodyAccJerk-mean()-X","fBodyAccJerk-mean()-Y","fBodyAccJerk-mean()-Z"]

pca = PCA(n_components=1)
pca.fit(np.array(train_set[features_to_remove]))
print("variance explained by new feature fBodyAccJerk-mean() : ",pca.explained_variance_ratio_)


y = pca.transform(np.array(train_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'fBodyAccJerk-mean()'})
train_set["fBodyAccJerk-mean()"] = new_component 

pca.fit(np.array(test_set[features_to_remove]))

y = pca.transform(np.array(test_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'fBodyAccJerk-mean()'})
test_set["fBodyAccJerk-mean()"] = new_component 

for feature in features_to_remove:
    if feature in train_set.columns:
        del train_set[feature]
    if feature in test_set.columns:
        del test_set[feature]

In [109]:
features_to_remove = ["tBodyAccJerk-std()-X","tBodyAccJerk-std()-Y","tBodyAccJerk-std()-Z"]

pca = PCA(n_components=1)
pca.fit(np.array(train_set[features_to_remove]))
print("variance explained by new feature tBodyAccJerk-std() : ",pca.explained_variance_ratio_)


y = pca.transform(np.array(train_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'tBodyAccJerk-std()'})
train_set["tBodyAccJerk-std()"] = new_component 

pca.fit(np.array(test_set[features_to_remove]))

y = pca.transform(np.array(test_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'tBodyAccJerk-std()'})
test_set["tBodyAccJerk-std()"] = new_component 

for feature in features_to_remove:
    if feature in train_set.columns:
        del train_set[feature]
    if feature in test_set.columns:
        del test_set[feature]

In [110]:
features_to_remove = ["tBodyAccJerk-mad()-X","tBodyAccJerk-mad()-Y","tBodyAccJerk-mad()-Z"]

pca = PCA(n_components=1)
pca.fit(np.array(train_set[features_to_remove]))
print("variance explained by new feature tBodyAccJerk-mad() : ",pca.explained_variance_ratio_)


y = pca.transform(np.array(train_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'tBodyAccJerk-mad()'})
train_set["tBodyAccJerk-mad()"] = new_component 

pca.fit(np.array(test_set[features_to_remove]))

y = pca.transform(np.array(test_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'tBodyAccJerk-mad()'})
test_set["tBodyAccJerk-mad()"] = new_component 

for feature in features_to_remove:
    if feature in train_set.columns:
        del train_set[feature]
    if feature in test_set.columns:
        del test_set[feature]

In [111]:
features_to_remove = ["tBodyAccJerk-max()-X","tBodyAccJerk-max()-Y","tBodyAccJerk-max()-Z"]

pca = PCA(n_components=1)
pca.fit(np.array(train_set[features_to_remove]))
print("variance explained by new feature tBodyAccJerk-max() : ",pca.explained_variance_ratio_)


y = pca.transform(np.array(train_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'tBodyAccJerk-max()'})
train_set["tBodyAccJerk-max()"] = new_component 

pca.fit(np.array(test_set[features_to_remove]))

y = pca.transform(np.array(test_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'tBodyAccJerk-max()'})
test_set["tBodyAccJerk-max()"] = new_component 

for feature in features_to_remove:
    if feature in train_set.columns:
        del train_set[feature]
    if feature in test_set.columns:
        del test_set[feature]

In [112]:
features_to_remove = ["tBodyAccJerk-entropy()-X","tBodyAccJerk-entropy()-Y","tBodyAccJerk-entropy()-Z"]

pca = PCA(n_components=1)
pca.fit(np.array(train_set[features_to_remove]))
print("variance explained by new feature tBodyAccJerk-entropy() : ",pca.explained_variance_ratio_)


y = pca.transform(np.array(train_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'tBodyAccJerk-entropy()'})
train_set["tBodyAccJerk-entropy()"] = new_component 

pca.fit(np.array(test_set[features_to_remove]))

y = pca.transform(np.array(test_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'tBodyAccJerk-entropy()'})
test_set["tBodyAccJerk-entropy()"] = new_component 

for feature in features_to_remove:
    if feature in train_set.columns:
        del train_set[feature]
    if feature in test_set.columns:
        del test_set[feature]

In [113]:
features_to_remove = ["fBodyAccMag-mean()","fBodyAccMag-std()","fBodyAccMag-mad()","fBodyAccMag-max()","fBodyAccMag-min()","fBodyAccMag-sma()","fBodyAccMag-energy()","fBodyAccMag-iqr()"]

pca = PCA(n_components=1)
pca.fit(np.array(train_set[features_to_remove]))
print("variance explained by new feature fBodyAccMag() : ",pca.explained_variance_ratio_)


y = pca.transform(np.array(train_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'fBodyAccMag'})
train_set["fBodyAccMag"] = new_component 

pca.fit(np.array(test_set[features_to_remove]))

y = pca.transform(np.array(test_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'fBodyAccMag'})
test_set["fBodyAccMag"] = new_component 

for feature in features_to_remove:
    if feature in train_set.columns:
        del train_set[feature]
    if feature in test_set.columns:
        del test_set[feature]

In [114]:
features_to_remove = ["tBodyGyroJerkMag-mean()","tBodyGyroJerkMag-std()","tBodyGyroJerkMag-mad()","tBodyGyroJerkMag-max()","tBodyGyroJerkMag-min()","tBodyGyroJerkMag-sma()","tBodyGyroJerkMag-energy()","tBodyGyroJerkMag-iqr()"]


pca = PCA(n_components=1)
pca.fit(np.array(train_set[features_to_remove]))
print("variance explained by new feature tBodyGyroJerkMag : ",pca.explained_variance_ratio_)

y = pca.transform(np.array(train_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'tBodyGyroJerkMag'})
train_set["tBodyGyroJerkMag"] = new_component 


y = pca.transform(np.array(test_set[features_to_remove]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'tBodyGyroJerkMag'})
test_set["tBodyGyroJerkMag"] = new_component 

for feature in features_to_remove:
    if feature in train_set.columns:
        del train_set[feature]
    if feature in test_set.columns:
        del test_set[feature]


there are still correlations but we will leave it as it's for now, features were designed by domain experts so it's better to not tweak them too much unless a some a large group of variable appear to be trivially corrolated

# Univariate analysis

a quick visualisation of the first tree variable, a pretty tigh interval of variations. The distribution looks vary similar to the gaussion normal distribution.

In [115]:
#tBodyAcc-mean()-X                       
#tBodyAcc-mean()-Y                       
#tBodyAcc-mean()-Z 
sns.distplot(a=train_set["tBodyAcc-mean()-X"], bins=40, color='purple',
             hist_kws={"edgecolor": 'black'})
 # visualizing plot using matplotlib.pyplot library
plt.show()

In [116]:
sns.distplot(a=train_set["tBodyAcc-mean()-Y"], bins=40, color='purple',
             hist_kws={"edgecolor": 'black'})
 # visualizing plot using matplotlib.pyplot library
plt.show()

In [117]:
sns.distplot(a=train_set["tBodyAcc-mean()-Z"], bins=40, color='purple',
             hist_kws={"edgecolor": 'black'})
 # visualizing plot using matplotlib.pyplot library
plt.show()

# Convert string activity features into numerical format

**one hot encoding**

We want to convert the target variable (activity) into a matrix of ints, using a one hot encoding format.

each row of the matrix refers to a binary vector that indicates  the activty, each index refers to its corresponding activity in the printed list.

In [118]:
labels = ["WALKING","WALKING_UPSTAIRS","WALKING_DOWNSTAIRS","SITTING","STANDING","LAYING"]
labels.sort()
print(labels)

In [119]:
enc = preprocessing.OneHotEncoder(handle_unknown='ignore')
encoder_train_data = pd.DataFrame(enc.fit_transform(train_set[["Activity"]]).toarray())
final_train_data = train_set.join(encoder_train_data)
del final_train_data["Activity"]
final_train_data.rename(columns = {0:'LAYING', 1:"SITTING",2:'STANDING',3:"WALKING",4:'WALKING_DOWNSTAIRS',5:"WALKING_UPSTAIRS"}, inplace = True)
y_train_one_hot = final_train_data[['LAYING', 'SITTING', 'STANDING', 'WALKING', 'WALKING_DOWNSTAIRS', 'WALKING_UPSTAIRS']].to_numpy()
del final_train_data['LAYING']
del final_train_data['SITTING']
del final_train_data['STANDING']
del final_train_data['WALKING']
del final_train_data['WALKING_DOWNSTAIRS']
del final_train_data['WALKING_UPSTAIRS']
x_train = final_train_data


In [120]:
encoder_test_data = pd.DataFrame(enc.fit_transform(test_set[["Activity"]]).toarray())
final_test_data = test_set.join(encoder_test_data)
del final_test_data["Activity"]
final_test_data.rename(columns = {0:'LAYING', 1:"SITTING",2:'STANDING',3:"WALKING",4:'WALKING_DOWNSTAIRS',5:"WALKING_UPSTAIRS"}, inplace = True)
y_test_one_hot = final_test_data[['LAYING', 'SITTING', 'STANDING', 'WALKING', 'WALKING_DOWNSTAIRS', 'WALKING_UPSTAIRS']].to_numpy()
del final_test_data['LAYING']
del final_test_data['SITTING']
del final_test_data['STANDING']
del final_test_data['WALKING']
del final_test_data['WALKING_DOWNSTAIRS']
del final_test_data['WALKING_UPSTAIRS']
x_test = final_test_data


In [121]:
print("training data size : features matrix ",x_train.shape," target matrix ",y_train_one_hot.shape)
print("testing data size : features matrix ",x_test.shape," target matrix ",y_test_one_hot.shape)

**ordinal encoding**

In [122]:
label_encoder = LabelEncoder()
label_encoder.fit(train_set["Activity"])
y_train_ordinal = label_encoder.transform(train_set["Activity"])
y_test_ordinal = label_encoder.transform(test_set["Activity"])


In [123]:
print("training data size : features matrix ",x_train.shape," 1D target vector",y_train_ordinal.shape)
print("testing data size : features matrix ",x_test.shape,"1D target vector",y_test_ordinal.shape)

# {0:'LAYING', 1:"SITTING",2:'STANDING',3:"WALKING",4:'WALKING_DOWNSTAIRS',5:"WALKING_UPSTAIRS"}

In [124]:
print(y_train_ordinal)

In [125]:
print(y_train_one_hot)

In [126]:
x_train_scaler = preprocessing.StandardScaler().fit(x_train)
x_train_scaled = x_train_scaler.transform(x_train)
x_test_scaler = preprocessing.StandardScaler().fit(x_test)
x_test_scaled = x_test_scaler.transform(x_test)

# Modeling

# SVM

**One vs One approach**

In [127]:
svc = svm.SVC(decision_function_shape='ovo',kernel='rbf')
start = time.time()
svc.fit(x_train,y_train_ordinal)
end = time.time()
print("training time complexity :",end-start,"s")
print("training accuracy :",svc.score(x_train,y_train_ordinal),"validation accuracy :",svc.score(x_test,y_test_ordinal))

In [128]:
svc = svm.SVC(decision_function_shape='ovo',kernel='linear')
start = time.time()
svc.fit(x_train,y_train_ordinal)
end = time.time()
print("training time complexity :",end-start,"s")
print("training accuracy :",svc.score(x_train,y_train_ordinal),"validation accuracy :",svc.score(x_test,y_test_ordinal))

In [129]:
def test_model(C_param):
    svc = svm.SVC(decision_function_shape='ovo',kernel='linear',C=C_param)
    start = time.time()
    svc.fit(x_train,y_train_ordinal)
    end = time.time()
    print("training time complexity :",end-start,"s")
    print("training accuracy :",svc.score(x_train,y_train_ordinal),"validation accuracy :",svc.score(x_test,y_test_ordinal))

In [130]:
C_param = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.1,1.2,1.3,1.5,1.6,1.7,1.8,1.9,2.0,3.0,4.0]
for c in C_param:
    print("C =",c)
    test_model(c)

In [139]:
svc = svm.SVC(decision_function_shape='ovo',kernel='linear',C=3)
start = time.time()
svc.fit(x_train,y_train_ordinal)
end = time.time()
print("training time complexity :",end-start,"s")
print("training accuracy :",svc.score(x_train,y_train_ordinal),"validation accuracy :",svc.score(x_test,y_test_ordinal))
y_pred = svc.predict(x_test)
cf_matrix = confusion_matrix(y_test_ordinal, y_pred)



# {0:'LAYING', 1:"SITTING",2:'STANDING',3:"WALKING",4:'WALKING_DOWNSTAIRS',5:"WALKING_UPSTAIRS"}

In [140]:
plt.figure(figsize=(20, 20))
ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues')


ax.set_title('Seaborn Confusion Matrix with labels for SVM model\n\n');
ax.set_xlabel('Predicted labels')
ax.set_ylabel('Actual labels ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(["LAYING","SITTING",'STANDING',"WALKING","WALKING_DOWNSTAIRS","WALKING_UPSTAIRS"])
ax.yaxis.set_ticklabels(["LAYING","SITTING",'STANDING',"WALKING","WALKING_DOWNSTAIRS","WALKING_UPSTAIRS"])


## Display the visualization of the Confusion Matrix.
plt.show()

**One vs rest approach**

In [143]:
svc = svm.SVC(decision_function_shape='ovr',kernel='linear',C=3.0)
start = time.time()
svc.fit(x_train,y_train_ordinal)
end = time.time()
print("training time complexity :",end-start,"s")
print("training accuracy :",svc.score(x_train,y_train_ordinal),"validation accuracy :",svc.score(x_test,y_test_ordinal))

# Logistic regression

In [144]:
Logreg = LogisticRegression(multi_class="multinomial",solver="saga")
start = time.time()
Logreg.fit(x_train,y_train_ordinal)
end = time.time()
print("training time complexity :",end-start,"s")
print("training accuracy :",Logreg.score(x_train,y_train_ordinal),"validation accuracy :",Logreg.score(x_test,y_test_ordinal))

Let's now use scaled data

In [145]:
Logreg = LogisticRegression(multi_class="multinomial",solver="saga")
start = time.time()
Logreg.fit(x_train_scaled,y_train_ordinal)
end = time.time()
print("training time complexity :",end-start,"s")
print("training accuracy :",Logreg.score(x_train_scaled,y_train_ordinal),"validation accuracy :",Logreg.score(x_test_scaled,y_test_ordinal))

In [146]:
Logreg = LogisticRegression(multi_class="multinomial",solver="newton-cg")
start = time.time()
Logreg.fit(x_train_scaled,y_train_ordinal)
end = time.time()
print("training time complexity :",end-start,"s")
print("training accuracy :",Logreg.score(x_train_scaled,y_train_ordinal),"validation accuracy :",Logreg.score(x_test_scaled,y_test_ordinal))

In [147]:
y_pred = Logreg.predict(x_test_scaled)
cf_matrix = confusion_matrix(y_test_ordinal, y_pred)
plt.figure(figsize=(20, 20))
ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues')


ax.set_title('Seaborn Confusion Matrix with labels for logistic regression model \n\n');
ax.set_xlabel('Predicted labels')
ax.set_ylabel('Actual labels ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(["LAYING","SITTING",'STANDING',"WALKING","WALKING_DOWNSTAIRS","WALKING_UPSTAIRS"])
ax.yaxis.set_ticklabels(["LAYING","SITTING",'STANDING',"WALKING","WALKING_DOWNSTAIRS","WALKING_UPSTAIRS"])


## Display the visualization of the Confusion Matrix.
plt.show()

# Perceptron

In [148]:
per = Perceptron(tol=1e-3, random_state=0,eta0=0.5,warm_start=True)
start = time.time()
per.fit(x_train_scaled,y_train_ordinal)
end = time.time()
print("training time complexity :",end-start,"s")
print("training accuracy :",per.score(x_train_scaled,y_train_ordinal),"validation accuracy :",per.score(x_test_scaled,y_test_ordinal))

In [151]:
# warm_start=True means the initial solution will be choosen as the previous solution provided by the last fit method call
per.fit(x_train,y_train_ordinal)
print("training accuracy :",per.score(x_train,y_train_ordinal),"validation accuracy :",per.score(x_test,y_test_ordinal))

In [152]:
y_pred = per.predict(x_test)
cf_matrix = confusion_matrix(y_test_ordinal, y_pred)
plt.figure(figsize=(20, 20))
ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues')


ax.set_title('Seaborn Confusion Matrix with labels for perceptron model model \n\n');
ax.set_xlabel('Predicted labels')
ax.set_ylabel('Actual labels ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(["LAYING","SITTING",'STANDING',"WALKING","WALKING_DOWNSTAIRS","WALKING_UPSTAIRS"])
ax.yaxis.set_ticklabels(["LAYING","SITTING",'STANDING',"WALKING","WALKING_DOWNSTAIRS","WALKING_UPSTAIRS"])


## Display the visualization of the Confusion Matrix.
plt.show()

# Synthesis :

Overall, the models did a good job in classifying activties, the models struglled the most in recognising WALKING, STANDING and SITTING ACTIVITES. In fact a small portion of the data points were misclassified. 41 data point refering to a person walking were classified as walking upstairs. The models confused some data points label, the confusion happened mostly between SITTING and STANDING records.

the two best performing models are the ones who resulted in the two accuracies : 0.963,0.959

Linear SVM with C = 3 0.963

Logistic regression optimized with newton solver 0.959